In [1]:
from OSLPP import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import os
import pandas as pd
from torch.optim.lr_scheduler import StepLR
import pickle

In [2]:
with open('small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [4]:
def get_abbrev(source, target): return source[0].upper() + target[0].upper()
def fmeasures(d): return ' '.join([f'{k}={v*100:.2f}' for (k,v) in d.items()])

In [5]:
class FeaturesDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    def __len__(self): return len(self.labels)
    def __getitem__(self, i): return self.features[i], self.labels[i]

In [6]:
def _create_balanced_sampler(ds:FeaturesDataset):
    freq2 = Counter(ds.labels.tolist())
    class_weight = {x: 1.0 / freq2[x] for x in freq2}
    source_weights = [class_weight[x] for x in ds.labels.tolist()]
    sampler = torch.utils.data.WeightedRandomSampler(source_weights, len(ds.labels.tolist()))
    return sampler

def _create_trn_dataloader(ds, batch_size, balanced):
    if balanced: return torch.utils.data.DataLoader(ds, batch_size=batch_size, sampler=_create_balanced_sampler(ds), drop_last=True)
    else: torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=True)

In [7]:
def train_initial_NN(feats_S, lbls_S, num_epochs, params, balanced, lr):
    num_src_classes = params.num_common + params.num_src_priv
    assert (lbls_S.unique() == torch.arange(num_src_classes)).all()
    model = nn.Sequential(nn.Linear(params.pca_dim, params.proj_dim), nn.ReLU(), nn.Linear(params.proj_dim, num_src_classes)).cuda().train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss().cuda()
    ds = FeaturesDataset(feats_S, lbls_S)
    dl = _create_trn_dataloader(ds, 32, balanced)
    for ep in range(num_epochs):
        for f,l in dl:
            optimizer.zero_grad()
            loss_fn(model(f.cuda()), l.cuda()).backward()
            optimizer.step()
    return model.eval()

In [8]:
def entropy_loss(logits):
    probs = F.softmax(logits, dim=1)
    entropy =  - probs * probs.log()
    entropy = entropy.sum(dim=1)
    return entropy.mean()

In [9]:
def train_NN(feats, lbls, num_epochs, params, balanced, lr):
    num_src_classes = params.num_common + params.num_src_priv
    feats, lbls = feats[lbls >= 0], lbls[lbls >= 0]
    feats, lbls = feats[lbls < num_src_classes + 1], lbls[lbls < num_src_classes + 1]
    model = nn.Sequential(nn.Linear(params.pca_dim, params.proj_dim), nn.ReLU(), nn.Linear(params.proj_dim, num_src_classes + 1)).cuda().train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss().cuda()
    ds = FeaturesDataset(feats, lbls)
    dl = _create_trn_dataloader(ds, 32, balanced)
    for ep in range(num_epochs):
        for f,l in dl:
            optimizer.zero_grad()
            loss_fn(model(f.cuda()), l.cuda()).backward()
            optimizer.step()
    return model.eval()

In [10]:
def predict_NN(model, feats, lbls):
    ds = FeaturesDataset(feats, lbls)
    dl = torch.utils.data.DataLoader(ds, batch_size=32, shuffle=False)
    model = model.eval()
    feats = []
    preds = []
    with torch.no_grad():
        for f,l in dl:
            out = model(f.cuda())
            preds.append(F.softmax(out, dim=1).detach().cpu())
            feats.append(f.detach())
    feats = torch.cat(feats, dim=0)
    preds = torch.cat(preds, dim=0)
    return feats, preds

In [11]:
def set_seed(seed:int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [12]:
results = pd.DataFrame({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': []})
# select_reject_mode = SelectRejectMode.ENTROPY
select_reject_mode = SelectRejectMode.CONFIDENCE
# select_reject_mode = SelectRejectMode.MARGIN
for (source, target), (common, tgt_private) in config.items():
    for epochs in [10, 50]:
        for lr in [1e-3, 1e-4]:
            for n_r in [0.1, 0.25, 0.35]:
                for seed in range(3):
                    set_seed(seed)
                    print(source, '->', target, 'lr=', lr, 'seed=', seed)
                    params = Params(pca_dim=512, proj_dim=128, T=10, n_r=1200, n_r_ratio=None,
                              dataset='DomainNet_DCC', source=source, target=target,
                              num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                    (feats_S, lbls_S), (feats_T, lbls_T) = create_datasets_sub(params.dataset, 
                                                                               params.source, 
                                                                               params.target, 
                                                                               common, 
                                                                               tgt_private)
                    params.n_r = int(len(lbls_T) * n_r)
                    num_src_classes = params.num_common + params.num_src_priv

                    # l2 normalization and pca
                    feats_S, feats_T = do_l2_normalization(feats_S, feats_T)
                    feats_S, feats_T = do_pca(feats_S, feats_T, params.pca_dim)
                    feats_S, feats_T = do_l2_normalization(feats_S, feats_T)

                    # initial
                    feats_S, feats_T = torch.tensor(feats_S), torch.tensor(feats_T)
                    lbls_S, lbls_T = torch.tensor(lbls_S), torch.tensor(lbls_T)
                    feats_all = torch.cat((feats_S, feats_T), dim=0)
                    pseudo_labels = -torch.ones_like(lbls_T)
                    rejected = torch.zeros_like(pseudo_labels)

                    t = 1

                    model = train_initial_NN(feats_S, lbls_S, epochs, params, balanced=True, lr=lr)
                    feats_S_2, preds_S_2 = predict_NN(model, feats_S, lbls_S)
                    feats_T_2, preds_T_2 = predict_NN(model, feats_T, lbls_T)
                    print((feats_S_2 == feats_S).all(), (feats_T_2 == feats_T).all())
                    assert (feats_S_2 == feats_S).all() and (feats_T_2 == feats_T).all()

                    if select_reject_mode == SelectRejectMode.CONFIDENCE:
                        confs, cs_pseudo_labels = preds_T_2.max(dim=1)
                        selected = select_closed_set_pseudo_labels(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, select_reject_mode)
                        selected = torch.tensor(selected)
                        rejected = select_initial_rejected(confs, params.n_r)
                    elif select_reject_mode == SelectRejectMode.ENTROPY:
                        confs, cs_pseudo_labels = preds_T_2.max(dim=1)
                        selected = select_closed_set_pseudo_labels(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, select_reject_mode)
                        selected = torch.tensor(selected)
                        rejected = select_initial_rejected_by_entropy(confs, params.n_r, preds_T_2)
                    elif select_reject_mode == SelectRejectMode.MARGIN:
                        confs, cs_pseudo_labels = preds_T_2.max(dim=1)
                        selected = select_closed_set_pseudo_labels(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, select_reject_mode)
                        selected = torch.tensor(selected)
                        rejected = select_initial_rejected_by_margin(confs, params.n_r, preds_T_2)
                    else:
                        raise Exception('Incorrect mode.')
                        
                    rejected = torch.tensor(rejected)
                    selected = selected * (1 - rejected)
                    
                    pseudo_labels = cs_pseudo_labels.clone()
                    pseudo_labels[rejected == 1] = num_src_classes
                    pseudo_labels[(rejected == 0) * (selected == 0)] = -1

                    metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, 
                            lbls_T.numpy(), cs_pseudo_labels.numpy(), rejected.numpy())
                    where = torch.where((selected == 1) + (rejected == 1))[0]
                    metrics_selected = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T[where].numpy(), cs_pseudo_labels[where].numpy(), rejected[where].numpy())
                    print('______')
                    print(f'Iteration t={t}')
                    print('all: ', fmeasures(metrics))
                    print('selected: ', fmeasures(metrics_selected))

                    for t in range(2, params.T):
                        model = train_NN(feats_all, torch.cat((lbls_S, pseudo_labels), axis=0), epochs, params, balanced=True, lr=lr)
                        feats_S_2, preds_S_2 = predict_NN(model, feats_S, lbls_S)
                        feats_T_2, preds_T_2 = predict_NN(model, feats_T, lbls_T)
                        assert (feats_S_2 == feats_S).all() and (feats_T_2 == feats_T).all()

                        confs, cs_pseudo_labels = preds_T_2.max(dim=1)
                        selected = torch.tensor(select_closed_set_pseudo_labels(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, select_reject_mode))
                        pseudo_labels = cs_pseudo_labels.clone()
                        pseudo_labels[selected == 0] = -1
                        print('Selected CNT:', len(selected[selected != 0]), '/', len(selected))

        
                        rejected = torch.zeros_like(pseudo_labels)
                        rejected[pseudo_labels == num_src_classes] = 1
            
                        metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T.numpy(), cs_pseudo_labels.numpy(), rejected.numpy())
                        where = torch.where((selected == 1) + (rejected == 1))[0]
                        metrics_selected = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T[where].numpy(), cs_pseudo_labels[where].numpy(), rejected[where].numpy())
                        print('______')
                        print(f'Iteration t={t}')
                        print('all: ', fmeasures(metrics))
                        print('selected: ', fmeasures(metrics_selected))

                    print((pseudo_labels == -1).sum())

                    _rejected = rejected
                    metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T.numpy(), cs_pseudo_labels.numpy(), _rejected.numpy())
                    print('all: ', fmeasures(metrics))

                    results = results.append({'source': source, 'target': target, 'desc': fmeasures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs}, ignore_index=True)

painting -> sketch lr= 0.001 seed= 0
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
tensor(True) tensor(True)
______
Iteration t=1
all:  cs/acc_i=50.56 cs/acc_c=52.02 os/recall_knw=94.12 os/recall_unk=17.15 total/acc_i=37.72 total/acc_c=49.41 total/h_score=25.67
selected:  cs/acc_i=57.90 cs/acc_c=59.11 os/recall_knw=72.57 os/recall_unk=82.51 total/acc_i=63.57 total/acc_c=55.95 total/h_score=65.73
Selected CNT: 2155 / 9659
______
Iteration t=2
all:  cs/acc_i=39.01 cs/acc_c=40.37 os/recall_knw=95.45 os/recall_unk=20.53 total/acc_i=49.83 total/acc_c=41.72 total/h_score=50.85
selected:  cs/acc_i=57.99 cs/acc_c=59.83 os/recall_knw=78.88 os/recall_unk=86.69 total/acc_i=69.10 total/acc_c=61.10 total/h_score=70.80
Selected CNT: 3227 / 9659
______
Iteration t=3
all:  cs/acc_i=42.76 cs/acc_c=44.07 os/recall_knw=94.61 os/recall_unk=22.15 total/acc_i=47.33 total/acc_c=44.60 total/h_score=49.05
selected:  cs/acc_i=59.45

Selected CNT: 7522 / 9659
______
Iteration t=7
all:  cs/acc_i=47.91 cs/acc_c=49.27 os/recall_knw=89.73 os/recall_unk=34.78 total/acc_i=46.01 total/acc_c=48.96 total/h_score=45.75
selected:  cs/acc_i=53.27 cs/acc_c=54.43 os/recall_knw=87.24 os/recall_unk=47.43 total/acc_i=51.26 total/acc_c=54.10 total/h_score=50.69
Selected CNT: 8595 / 9659
______
Iteration t=8
all:  cs/acc_i=48.57 cs/acc_c=49.97 os/recall_knw=89.12 os/recall_unk=36.17 total/acc_i=45.44 total/acc_c=49.50 total/h_score=44.42
selected:  cs/acc_i=51.19 cs/acc_c=52.58 os/recall_knw=87.99 os/recall_unk=41.98 total/acc_i=47.93 total/acc_c=52.07 total/h_score=46.68
Selected CNT: 9659 / 9659
______
Iteration t=9
all:  cs/acc_i=48.85 cs/acc_c=50.27 os/recall_knw=87.45 os/recall_unk=40.23 total/acc_i=45.71 total/acc_c=49.80 total/h_score=44.70
selected:  cs/acc_i=48.85 cs/acc_c=50.27 os/recall_knw=87.45 os/recall_unk=40.23 total/acc_i=45.71 total/acc_c=49.80 total/h_score=44.70
tensor(0)
all:  cs/acc_i=48.85 cs/acc_c=50.27 os/rec

Selected CNT: 4304 / 9659
______
Iteration t=4
all:  cs/acc_i=43.98 cs/acc_c=45.39 os/recall_knw=92.41 os/recall_unk=26.60 total/acc_i=47.10 total/acc_c=45.73 total/h_score=48.70
selected:  cs/acc_i=57.34 cs/acc_c=58.41 os/recall_knw=84.06 os/recall_unk=67.85 total/acc_i=60.71 total/acc_c=58.86 total/h_score=62.78
Selected CNT: 5374 / 9659
______
Iteration t=5
all:  cs/acc_i=45.38 cs/acc_c=46.69 os/recall_knw=91.62 os/recall_unk=28.68 total/acc_i=45.94 total/acc_c=46.70 total/h_score=46.80
selected:  cs/acc_i=56.59 cs/acc_c=57.60 os/recall_knw=85.82 os/recall_unk=57.78 total/acc_i=56.98 total/acc_c=57.60 total/h_score=57.69
Selected CNT: 6444 / 9659
______
Iteration t=6
all:  cs/acc_i=47.22 cs/acc_c=48.57 os/recall_knw=90.76 os/recall_unk=31.20 total/acc_i=45.86 total/acc_c=48.33 total/h_score=45.89
selected:  cs/acc_i=55.39 cs/acc_c=56.49 os/recall_knw=86.85 os/recall_unk=51.52 total/acc_i=54.11 total/acc_c=56.26 total/h_score=53.89
Selected CNT: 7522 / 9659
______
Iteration t=7
all: 

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
tensor(True) tensor(True)
______
Iteration t=1
all:  cs/acc_i=50.35 cs/acc_c=51.97 os/recall_knw=73.73 os/recall_unk=50.20 total/acc_i=45.63 total/acc_c=44.77 total/h_score=47.17
selected:  cs/acc_i=43.85 cs/acc_c=45.81 os/recall_knw=37.01 os/recall_unk=92.95 total/acc_i=54.67 total/acc_c=30.96 total/h_score=42.87
Selected CNT: 2152 / 9659
______
Iteration t=2
all:  cs/acc_i=34.04 cs/acc_c=34.96 os/recall_knw=93.56 os/recall_unk=23.99 total/acc_i=50.19 total/acc_c=37.02 total/h_score=48.34
selected:  cs/acc_i=50.24 cs/acc_c=52.00 os/recall_knw=67.89 os/recall_unk=91.65 total/acc_i=67.98 total/acc_c=53.88 total/h_score=66.35
Selected CNT: 3226 / 9659
______
Iteration t=3
all:  cs/acc_i=40.07 cs/acc_c=41.21 os/recall_knw=92.49 os/recall_unk=26.21 total/acc_i=48.78 total/acc_c=42.29 total/h_score=50.13
selected:  cs/acc_i=57.48 cs/acc_c=58.89 os/recall_knw=78.03 o

Selected CNT: 7520 / 9659
______
Iteration t=7
all:  cs/acc_i=46.26 cs/acc_c=47.94 os/recall_knw=84.93 os/recall_unk=18.11 total/acc_i=38.62 total/acc_c=46.86 total/h_score=33.12
selected:  cs/acc_i=50.70 cs/acc_c=52.00 os/recall_knw=81.83 os/recall_unk=26.27 total/acc_i=42.81 total/acc_c=50.77 total/h_score=34.90
Selected CNT: 8597 / 9659
______
Iteration t=8
all:  cs/acc_i=46.75 cs/acc_c=48.52 os/recall_knw=82.09 os/recall_unk=21.41 total/acc_i=38.92 total/acc_c=47.41 total/h_score=33.23
selected:  cs/acc_i=49.10 cs/acc_c=50.49 os/recall_knw=80.53 os/recall_unk=25.53 total/acc_i=41.00 total/acc_c=49.30 total/h_score=33.92
Selected CNT: 9659 / 9659
______
Iteration t=9
all:  cs/acc_i=46.96 cs/acc_c=48.74 os/recall_knw=80.56 os/recall_unk=25.04 total/acc_i=38.97 total/acc_c=47.61 total/h_score=33.08
selected:  cs/acc_i=46.96 cs/acc_c=48.74 os/recall_knw=80.56 os/recall_unk=25.04 total/acc_i=38.97 total/acc_c=47.61 total/h_score=33.08
tensor(0)
all:  cs/acc_i=46.96 cs/acc_c=48.74 os/rec

...

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
tensor(True) tensor(True)
______
Iteration t=1
all:  cs/acc_i=59.92 cs/acc_c=59.92 os/recall_knw=71.12 os/recall_unk=48.05 total/acc_i=50.41 total/acc_c=51.70 total/h_score=49.89
selected:  cs/acc_i=50.81 cs/acc_c=51.44 os/recall_knw=34.40 os/recall_unk=94.33 total/acc_i=53.71 total/acc_c=36.03 total/h_score=49.03
Selected CNT: 1936 / 8679
______
Iteration t=2
all:  cs/acc_i=37.64 cs/acc_c=38.14 os/recall_knw=90.88 os/recall_unk=24.67 total/acc_i=52.17 total/acc_c=40.29 total/h_score=52.31
selected:  cs/acc_i=53.07 cs/acc_c=52.70 os/recall_knw=55.31 os/recall_unk=93.56 total/acc_i=68.34 total/acc_c=54.64 total/h_score=67.42
Selected CNT: 2899 / 8679
______
Iteration t=3
all:  cs/acc_i=47.91 cs/acc_c=47.82 os/recall_knw=91.27 os/recall_unk=25.83 total/acc_i=52.93 total/acc_c=48.57 total/h_score=54.61
selected:  cs/acc_i=67.45 cs/acc_c=66.01 os/recall_knw=74.59 o

Selected CNT: 6760 / 8679
______
Iteration t=7
all:  cs/acc_i=53.83 cs/acc_c=53.54 os/recall_knw=87.09 os/recall_unk=33.78 total/acc_i=49.89 total/acc_c=52.96 total/h_score=46.74
selected:  cs/acc_i=59.93 cs/acc_c=59.18 os/recall_knw=83.75 os/recall_unk=45.26 total/acc_i=55.44 total/acc_c=58.52 total/h_score=51.29
Selected CNT: 7724 / 8679
______
Iteration t=8
all:  cs/acc_i=54.37 cs/acc_c=54.05 os/recall_knw=85.30 os/recall_unk=36.56 total/acc_i=49.96 total/acc_c=53.41 total/h_score=46.33
selected:  cs/acc_i=57.15 cs/acc_c=56.69 os/recall_knw=83.66 os/recall_unk=42.08 total/acc_i=52.46 total/acc_c=56.00 total/h_score=48.30
Selected CNT: 8679 / 8679
______
Iteration t=9
all:  cs/acc_i=54.36 cs/acc_c=54.06 os/recall_knw=83.71 os/recall_unk=39.63 total/acc_i=49.66 total/acc_c=53.37 total/h_score=45.73
selected:  cs/acc_i=54.36 cs/acc_c=54.06 os/recall_knw=83.71 os/recall_unk=39.63 total/acc_i=49.66 total/acc_c=53.37 total/h_score=45.73
tensor(0)
all:  cs/acc_i=54.36 cs/acc_c=54.06 os/rec

Selected CNT: 3867 / 8679
______
Iteration t=4
all:  cs/acc_i=63.63 cs/acc_c=63.44 os/recall_knw=91.95 os/recall_unk=20.45 total/acc_i=57.94 total/acc_c=62.60 total/h_score=53.20
selected:  cs/acc_i=77.75 cs/acc_c=76.64 os/recall_knw=84.76 os/recall_unk=76.08 total/acc_i=77.42 total/acc_c=76.62 total/h_score=76.36
Selected CNT: 4830 / 8679
______
Iteration t=5
all:  cs/acc_i=64.47 cs/acc_c=64.31 os/recall_knw=90.90 os/recall_unk=22.25 total/acc_i=56.76 total/acc_c=63.16 total/h_score=49.53
selected:  cs/acc_i=76.12 cs/acc_c=75.16 os/recall_knw=86.02 os/recall_unk=62.73 total/acc_i=73.40 total/acc_c=74.57 total/h_score=68.39
Selected CNT: 5791 / 8679
______
Iteration t=6
all:  cs/acc_i=65.66 cs/acc_c=65.56 os/recall_knw=90.15 os/recall_unk=24.35 total/acc_i=56.44 total/acc_c=64.18 total/h_score=47.09
selected:  cs/acc_i=74.14 cs/acc_c=73.53 os/recall_knw=86.95 os/recall_unk=50.60 total/acc_i=68.73 total/acc_c=72.44 total/h_score=59.95
Selected CNT: 6759 / 8679
______
Iteration t=7
all: 

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
tensor(True) tensor(True)
______
Iteration t=1
all:  cs/acc_i=67.55 cs/acc_c=68.21 os/recall_knw=83.01 os/recall_unk=42.09 total/acc_i=56.54 total/acc_c=63.03 total/h_score=50.81
selected:  cs/acc_i=59.39 cs/acc_c=62.57 os/recall_knw=47.93 os/recall_unk=95.88 total/acc_i=65.51 total/acc_c=53.16 total/h_score=66.61
Selected CNT: 1936 / 8679
______
Iteration t=2
all:  cs/acc_i=53.59 cs/acc_c=54.21 os/recall_knw=93.45 os/recall_unk=21.13 total/acc_i=61.47 total/acc_c=55.36 total/h_score=64.06
selected:  cs/acc_i=69.64 cs/acc_c=71.02 os/recall_knw=70.77 os/recall_unk=95.59 total/acc_i=77.84 total/acc_c=72.19 total/h_score=81.49
Selected CNT: 2899 / 8679
______
Iteration t=3
all:  cs/acc_i=58.37 cs/acc_c=58.60 os/recall_knw=92.01 os/recall_unk=20.16 total/acc_i=57.75 total/acc_c=58.49 total/h_score=57.49
selected:  cs/acc_i=76.18 cs/acc_c=75.89 os/recall_knw=79.22 o

Selected CNT: 6758 / 8679
______
Iteration t=7
all:  cs/acc_i=65.86 cs/acc_c=65.99 os/recall_knw=88.56 os/recall_unk=28.32 total/acc_i=56.58 total/acc_c=64.60 total/h_score=47.23
selected:  cs/acc_i=71.70 cs/acc_c=71.58 os/recall_knw=86.40 os/recall_unk=43.87 total/acc_i=64.34 total/acc_c=70.27 total/h_score=54.40
Selected CNT: 7725 / 8679
______
Iteration t=8
all:  cs/acc_i=66.74 cs/acc_c=66.96 os/recall_knw=88.02 os/recall_unk=30.09 total/acc_i=56.29 total/acc_c=65.39 total/h_score=45.06
selected:  cs/acc_i=69.80 cs/acc_c=69.69 os/recall_knw=87.03 os/recall_unk=36.73 total/acc_i=60.09 total/acc_c=68.12 total/h_score=48.10
Selected CNT: 8679 / 8679
______
Iteration t=9
all:  cs/acc_i=67.11 cs/acc_c=67.38 os/recall_knw=87.28 os/recall_unk=32.30 total/acc_i=56.01 total/acc_c=65.71 total/h_score=43.67
selected:  cs/acc_i=67.11 cs/acc_c=67.38 os/recall_knw=87.28 os/recall_unk=32.30 total/acc_i=56.01 total/acc_c=65.71 total/h_score=43.67
tensor(0)
all:  cs/acc_i=67.11 cs/acc_c=67.38 os/rec

In [13]:
results.to_csv('results/dcc__conf__small__NN_N+1__nn_raw.csv', header=True, index=False)

In [20]:
# df = pd.read_csv('office_home_oslpp_nn_raw.csv')
# df['h_score'] = df['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))
# import matplotlib.pyplot as plt
# for n, gr in df.groupby(['source', 'target']):
#     stat = gr.groupby(['lr', 'n_r'])['h_score'].mean()
#     exps = np.array(list(stat.index))
#     fig, ax = plt.subplots()
#     ax.scatter(exps[:, 0], exps[:, 1], c=stat.values)
#     for i, txt in enumerate(stat.values):
#         ax.annotate(round(txt, 2), (exps[i][0], exps[i][1]))
#     plt.title(n)
#     plt.xlabel('lr')
#     plt.ylabel('n_r')
#     plt.xticks(exps[:, 0])
#     plt.yticks(exps[:, 1])
#     fig.colorbar(ax.get_children()[0])
#     plt.show()

In [15]:
results

,source,target,desc,lr,seed,epochs,n_r
0,painting,sketch,cs/acc_i=48.70 cs/acc_c=50.07 os/recall_knw=87...,0.0010,0.0,10.0,0.10
1,painting,sketch,cs/acc_i=48.53 cs/acc_c=49.85 os/recall_knw=88...,0.0010,1.0,10.0,0.10
2,painting,sketch,cs/acc_i=48.85 cs/acc_c=50.27 os/recall_knw=87...,0.0010,2.0,10.0,0.10
3,painting,sketch,cs/acc_i=48.82 cs/acc_c=50.38 os/recall_knw=87...,0.0010,0.0,10.0,0.25
4,painting,sketch,cs/acc_i=48.18 cs/acc_c=49.44 os/recall_knw=86...,0.0010,1.0,10.0,0.25
...,...,...,...,...,...,...,...
211,sketch,painting,cs/acc_i=66.62 cs/acc_c=66.92 os/recall_knw=87...,0.0001,1.0,50.0,0.25
212,sketch,painting,cs/acc_i=65.93 cs/acc_c=66.13 os/recall_knw=86...,0.0001,2.0,50.0,0.25
213,sketch,painting,cs/acc_i=66.50 cs/acc_c=66.73 os/recall_knw=86...,0.0001,0.0,50.0,0.35
214,sketch,painting,cs/acc_i=67.11 cs/acc_c=67.38 os/recall_knw=87...,0.0001,1.0,50.0,0.35


In [30]:
results['epoch'] = pd.Series(results.index).apply(lambda i: epochs[(i // 9) % 3])

In [16]:
wd = results.copy()
wd['h_score'] = wd['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))

In [17]:
wd

,source,target,desc,lr,seed,epochs,n_r,h_score
0,painting,sketch,cs/acc_i=48.70 cs/acc_c=50.07 os/recall_knw=87...,0.0010,0.0,10.0,0.10,44.19
1,painting,sketch,cs/acc_i=48.53 cs/acc_c=49.85 os/recall_knw=88...,0.0010,1.0,10.0,0.10,42.12
2,painting,sketch,cs/acc_i=48.85 cs/acc_c=50.27 os/recall_knw=87...,0.0010,2.0,10.0,0.10,44.70
3,painting,sketch,cs/acc_i=48.82 cs/acc_c=50.38 os/recall_knw=87...,0.0010,0.0,10.0,0.25,42.28
4,painting,sketch,cs/acc_i=48.18 cs/acc_c=49.44 os/recall_knw=86...,0.0010,1.0,10.0,0.25,44.69
...,...,...,...,...,...,...,...,...
211,sketch,painting,cs/acc_i=66.62 cs/acc_c=66.92 os/recall_knw=87...,0.0001,1.0,50.0,0.25,42.87
212,sketch,painting,cs/acc_i=65.93 cs/acc_c=66.13 os/recall_knw=86...,0.0001,2.0,50.0,0.25,42.61
213,sketch,painting,cs/acc_i=66.50 cs/acc_c=66.73 os/recall_knw=86...,0.0001,0.0,50.0,0.35,45.17
214,sketch,painting,cs/acc_i=67.11 cs/acc_c=67.38 os/recall_knw=87...,0.0001,1.0,50.0,0.35,43.67


In [18]:
wd.groupby(['source', 'target', 'lr', 'n_r', 'epochs'])['h_score'].mean().to_frame(name = 'h_score').reset_index().sort_values(by='h_score', ascending=False).drop_duplicates(['source', 'target'])

,source,target,lr,n_r,epochs,h_score
70,sketch,real,0.001,0.35,10.0,69.830000
11,painting,real,0.001,0.35,50.0,68.933333
35,real,painting,0.001,0.35,50.0,59.510000
43,real,sketch,0.001,0.10,50.0,48.450000
58,sketch,painting,0.001,0.35,10.0,46.916667
23,painting,sketch,0.001,0.35,50.0,46.120000


In [19]:
wd.pivot(index=['lr', 'epochs', 'n_r'], columns=['source', 'target'], values='h_score')

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
wd.set_index(['lr', 'epoch', 'n_r']).head(30)

In [44]:
wd = pd.read_csv('results/dcc__entropy__StepLR__50_epochs__nn_raw.csv')

In [45]:
wd['h_score'] = wd['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))

In [46]:
wd

,source,target,desc,lr,seed,n_r,h_score
0,painting_train,real_train,cs/acc_i=59.98 cs/acc_c=60.09 os/recall_knw=71...,0.0005,0.0,0.15,54.40
1,painting_train,sketch_train,cs/acc_i=26.40 cs/acc_c=25.41 os/recall_knw=53...,0.0005,0.0,0.15,30.42
2,real_train,painting_train,cs/acc_i=39.74 cs/acc_c=39.39 os/recall_knw=65...,0.0005,0.0,0.15,40.83
3,real_train,sketch_train,cs/acc_i=24.48 cs/acc_c=23.68 os/recall_knw=51...,0.0005,0.0,0.15,29.29
4,sketch_train,painting_train,cs/acc_i=37.51 cs/acc_c=34.45 os/recall_knw=70...,0.0005,0.0,0.15,35.06
5,sketch_train,real_train,cs/acc_i=56.05 cs/acc_c=57.00 os/recall_knw=71...,0.0005,0.0,0.15,48.56
